# Qdrant with a Docker Container

In a terminal do the following

```
docker run -p 6333:6333 -p 6334:6334 qdrant/qdrant

```

Qdrant dashboard should be available at http://localhost:6333/dashboard


## Imports

In [1]:
from haystack import Document, Pipeline
from haystack.components.writers import DocumentWriter
from haystack_integrations.components.retrievers.qdrant import QdrantHybridRetriever
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack.document_stores.types import DuplicatePolicy
from haystack_integrations.components.embedders.fastembed import (
 FastembedTextEmbedder,
 FastembedDocumentEmbedder,
 FastembedSparseTextEmbedder,
 FastembedSparseDocumentEmbedder
)

/home/asheller/haystack/lib/python3.10/site-packages/haystack/core/errors.py:34: DeprecationWarning: PipelineMaxLoops is deprecated and will be remove in version '2.7.0'; use PipelineMaxComponentRuns instead.
  warnings.warn(


## Indexing

In [2]:
%%time
document_store = QdrantDocumentStore(host="localhost",
                                     index="ex02_docs",
                                     embedding_dim=384,
                                     recreate_index=True,
                                     use_sparse_embeddings=True,
                                     timeout=120
                                    )

documents = [
    Document(content="My name is Wolfgang and I live in Berlin"),
    Document(content="I saw a black horse running"),
    Document(content="Germany has many big cities"),
    Document(content="fastembed is supported by and maintained by Qdrant."),
]

indexing = Pipeline()
indexing.add_component("sparse_doc_embedder", FastembedSparseDocumentEmbedder(model="prithvida/Splade_PP_en_v1"))
indexing.add_component("dense_doc_embedder", FastembedDocumentEmbedder(model="BAAI/bge-small-en-v1.5"))
indexing.add_component("writer", DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE))
indexing.connect("sparse_doc_embedder", "dense_doc_embedder")
indexing.connect("dense_doc_embedder", "writer")

indexing.run({"sparse_doc_embedder": {"documents": documents}})

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Calculating embeddings: 100%|████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 104.99it/s]
100it [00:00, 10799.49it/s]                                                                                             

CPU times: user 1.97 s, sys: 313 ms, total: 2.29 s
Wall time: 1.1 s


{'writer': {'documents_written': 4}}

## Querying

In [3]:
querying = Pipeline()
querying.add_component("sparse_text_embedder", FastembedSparseTextEmbedder(model="prithvida/Splade_PP_en_v1"))
querying.add_component("dense_text_embedder", FastembedTextEmbedder(
 model="BAAI/bge-small-en-v1.5", prefix="Represent this sentence for searching relevant passages: ")
 )
querying.add_component("retriever", QdrantHybridRetriever(document_store=document_store))

querying.connect("sparse_text_embedder.sparse_embedding", "retriever.query_sparse_embedding")
querying.connect("dense_text_embedder.embedding", "retriever.query_embedding")

question = "Who supports fastembed?"

results = querying.run(
    {"dense_text_embedder": {"text": question},
     "sparse_text_embedder": {"text": question}}
)

print(results["retriever"]["documents"][0])


Calculating embeddings: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 36.31it/s]

Document(id=09074d8d1e7a956dcfa8a0cc75cabad565448c6701e92dc41314eae07af56265, content: 'fastembed is supported by and maintained by Qdrant.', score: 1.0)
